# Fundamentos de Ciência de Dados

Work made by:
- João António, nº 76558, joaoantonio@ua.pt, Mestrado em ciência de dados (9306)
- Tiago Freitas, nº 76748, tiagofreitas79@ua.pt, Mestrado em ciência de dados (9306)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MaxAbsScaler
import json
from plotly import graph_objects as go

%matplotlib inline

## 1. INE Cultures data

### 1.1. Information on INE data
Organizações não governamentais de ambiente por 100 000 habitantes (N.º) por Localização geográfica (NUTS - 2013); Anual
https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_indicadores&indOcorrCod=0008290&contexto=bd&selTab=tab2

Base de dados: Nuts 2013


Sinais convencionais:
Sinais convencionais por ausência de valor
- `…`	Dado confidencial
- `-`	Dado nulo ou não aplicável
- `x`	Dado não disponível
- `o`	Dado inferior a metade do módulo da unidade utilizada
- `∞`	Infinito

Sinais convencionais adicionais ao valor
- `//` Dado preliminar
- `&`	Dado provisório
- `»`	Dado previsto
- `*`	Dado rectificado
- `§`	Desvio do padrão de qualidade/Coeficiente de variação elevado
- `“`	Estimativa
- `“E` Dado estimado pelo Eurostat
- `┴`	Quebra de série/comparabilidade
- `i`	Mais informação em anexo

Sinais convencionais diversos:
- `µ`	Média
- `=`	Igual
- `>`	Maior que
- `≥`	Maior ou igual
- `<`	Menor que
- `≤`	Menor ou igual
- `%`	Percentagem
- `‰`	Permilagem
- `∑`	Soma de
- `≠`	Diferente

### 1.2. Data Load and Processing

In [ ]:
file_csv = "./data_FCD/ine_principais_culturas_agricolas.csv"

In [ ]:
# Data relating to Portugal map
with open("new_data/Portugal.json", "r") as read_file:
    world_geo = json.load(read_file)

In [ ]:
def load_ine_cultures(file_csv):
    df=pd.read_csv(file_csv,delimiter=";", encoding="latin-1", header=[4, 6], nrows=12, on_bad_lines="skip")    

    ## Drop Columns
    df.drop(labels=[0], inplace=True) # Drop first row
    df.drop(columns=df.columns[-1], inplace=True) # Drop last column
    
    df.drop(labels=[9,11], inplace=True) # Drop repeated Açores and Madeira rows
    
    ## Reset of the Index as Region with just the region codes
    df['Region_split'] = df[df.columns[0]].apply(lambda x : str(x).rsplit(sep = ':', maxsplit=1))
    df['Region']= df['Region_split'].apply(lambda x : np.nan if x[0] == 'nan' else x[0])
    df['Region_names']= df['Region_split'].apply(lambda x : np.nan if x[0] == 'nan' else x[1])
    region_names = df["Region_names"].to_list()
    df.set_index("Region", inplace=True)
    columns_to_rem = [0, -1, -2] # Remove Unnamed: 0_level_1(1st), Region_split(last) and Region_names(before last) columns
    df.drop(df.columns[columns_to_rem], axis=1, inplace=True)
    
    ## Changed regions names to just one word per region to make it easier for graphical representations later
    for k_ind, k in enumerate(region_names):
        k_split = k.split(" ")
        if len(k_split) > 1:
            region_names[k_ind] = k_split[-1]
    
    ## Redo the Headers, from unnamed to the year
    c0 = df.columns.get_level_values(0).to_series()
    c1 = list(df.columns.get_level_values(1).to_series())
    c0 = c0.mask(lambda c: c.str.startswith("Unnamed")).ffill()
    df.columns = [c0, c1]
    
    ## Bringing the year to the index
    df = df.stack(level=0)
    df.index.names=['Region', 'Year']
    
    ## Replacing 'x', 'x x', '- -' for their respective value
    df.replace({'x':np.nan,'x x':np.nan,'- -':0,'nan':np.nan},inplace=True)
    
    ## Replacing provisional data 
    for column in df:
        df[column]=df[column].map(lambda x: str(x).rstrip(' &'))
        df[column]=df[column].map(lambda x: str(x).rstrip(' *'))
        
    # This method turns data to stringd so we have to turn the data back to numeric
    for column in df:
        df[column]=pd.to_numeric(df[column],'coerce')
    
    return df, region_names

In [ ]:
data, region_names = load_ine_cultures(file_csv)
data

## 2. Data Visualization

### 2.1. Production by Region for a specific year

In [ ]:
plt.figure(figsize=(16, 8))
df_2020=data.xs("2020", level=1)

df_2020.columns
## Obtain index of Specific Products draw those index on plot
labels_to_plot = ["Trigo", "Arroz", "Amêndoa", "Trigo mole", "Trigo duro", "Uva para vinho"]
index_to_plot = [list(df_2020.columns).index(product) for product in labels_to_plot]

# filter_col = ["Trigo", "Trigo duro", "Ananás", "Amêndoa",]
df_2020[df_2020.columns[index_to_plot]].plot(figsize=(16, 6), marker="o")
plt.title("Quantity of produced agriculture products (in kg/ha) by region in 2020")
plt.xlabel("Regions")
plt.ylabel("Quantity produced in kg/ ha")
plt.legend(loc="upper left", ncol=2)
plt.grid()
plt.xticks(range(len(region_names)), region_names)
plt.show()
# plt.savefig("Production_2020.png")

### 2.2. Map of production for a given year and product

In [ ]:
def map_date(product, year_selected, colorscale="rainbow", meteo=False):
    if meteo:
        df_year=meteo.xs(year_selected, level=1)
    else:
        df_year=data.xs(year_selected, level=1)
    d = {
        region: df_year[product][df_year[product].index[k]]
        for k, region in enumerate(region_names)
    }
    df_product_region = pd.DataFrame(d.items(), columns=['City/County', 'Values'])

    df_product_region = df_product_region.set_index("City/County")


    new_d = {}
    for k in range(20):
        name = world_geo["features"][k]["properties"]["name"]
        region = world_geo["features"][k]["properties"]["region"]
        new_d[name] = df_product_region.loc[region][0]

    df_name =  pd.DataFrame(new_d.items(), columns=['City/County', 'Values'])
    #Create figure object
    fig = go.Figure(
        go.Choroplethmapbox(
            geojson = "new_data/Portugal.json", #Assign geojson file
            featureidkey = "properties.name", #Assign feature key
            locations = df_name["City/County"], #Assign location data
            z = df_name["Values"], #Assign information data
            zauto = True,
            colorscale = colorscale,
            showscale = True,
        )
    )


    #Update layout
    fig.update_layout(
        mapbox_style = "carto-positron", #Decide a style for the map
        mapbox_zoom = 3.8, #Zoom in scale
        mapbox_center = {"lat": 37.5, "lon": -17}, #Center location of the map
    )
        

In [ ]:
year_selected = "2020"
product = "Arroz"
df_name = map_date(product, year_selected)

### 2.3. Boxplot of productions by region

In [ ]:
def boxplot_data():
    data_regions = data.reset_index()
    return data_regions.set_index("Year")

In [ ]:
sns.set(rc = {'figure.figsize':(15,5)})
data_regions = boxplot_data()
sns.boxplot(data=data_regions, x="Region", y="Milho")
plt.xticks(range(len(region_names)), region_names)

In [ ]:
sns.set(rc = {'figure.figsize':(15,5)})
sns.boxplot(data=data_regions, x="Region", y="Tomate para indústria")
plt.xticks(range(len(region_names)), region_names)

### 2.4. Evolution of production
The main question was eher to compare how the products production evolved overtime using the 2000 as a baseline, and review the dataset for every 5 years thereafter.

In [ ]:
baseline_year = 2000

def evolution_baseline(region, baseline_year):
    def product_base_comp(region, year_study):
        data_region = data.xs(str(region))
        year_index = list(data_region.index).index(str(year_study))
        return data_region.iloc[year_index]

    d1 = pd.DataFrame()
    for year in [2000, 2005, 2010, 2015, 2020]:
        d1[year] = product_base_comp(region, year)

    d1["temp"] = d1[baseline_year]
    for year in [2000, 2005, 2010, 2015, 2020]:
        d1[year] /= d1["temp"]

    d1.dropna(axis = 0, how = 'all', inplace = True)
    d1.drop("temp", axis=1, inplace=True)
    return d1

In [ ]:
d1 = evolution_baseline(11, baseline_year)
sns.set(rc = {'figure.figsize':(15,5)})
sns.scatterplot(data=d1, palette='rocket_r')
plt.rcParams["xtick.labelsize"] = 10
plt.xticks(rotation=90)
plt.show()

In [ ]:
d2 = evolution_baseline(18, baseline_year)
sns.set(rc = {'figure.figsize':(15,5)})
d2.drop('Figo', axis=0, inplace=True)
sns.scatterplot(data=d2, palette='rocket_r')
plt.rcParams["xtick.labelsize"] = 10
plt.xticks(rotation=90)
plt.show()

On the 2 graphs above we can see how the production changed from the 2000 to 2005, 2010, 2015, 2020.
On the second graph (region 18) we decided to remove the Figo, since it made a giant leap in production (20x), making it harder to see all the other products evolution.

By comparing the 2 graphs, the region 18 seems to evolve more leading a better production overall, while the region 11 seems to have decrease the production.

## 3. IPMA Data

### 3.1 Loading IPMA Data
Note that the Structure of pages inside the Excel is given by:
- Shet 0 is metadata
- Shet 1 is tmin
- Shet 2 is tmax
- Shet 3 is Pluviosity

In [ ]:
# Global Functions have the name in full capslock to respect pep8
# 1st Code is of the region, the second is the name of the IPMA
MAP_REGIONS_EXCEL = {
    "Norte": [11, 11],
    "Centro": [16, 132],
    "Lisboa": [17, 320],
    "Alentejo": [18, 571],
    "Algarve": [15, 554],
    "Açores": [2, 360],
    "Madeira": [3, 522],
}
IPMA_COLUMNS = ["Tmin", "Tmax", "Prec", "TmaxA", "TminA", "Prec_max", "Prec_min"]

In [ ]:
def add_extra_col(meteo_coln, sheet_number):
    sheet_number += 1  # So that 1 is the Tmin, 2 is the Tmax and 3 is the precipitation
    index_describe = 1  # Mean

    if meteo_coln == "TmaxA":
        # Meaning T Max Ablsolute, so the real maximum temperature that we obtained that year
        index_describe = 7  # Max
        sheet_number = 2  # Tmax

    elif meteo_coln == "TminA":
        # Meaning T Min Ablsolute, so the real minimum temperature that we obtained that year
        index_describe = 3  # Min
        sheet_number = 1  # Tmin

    elif meteo_coln == "Prec_min":
        # Meaning T Min Ablsolute, so the real minimum temperature that we obtained that year
        index_describe = 3  # Min
        sheet_number = 3  # Precipitation Sheet

    elif meteo_coln == "Prec_max":
        # Meaning T Min Ablsolute, so the real minimum temperature that we obtained that year
        index_describe = 7  # Max
        sheet_number = 3  # Precipitation Sheet

    for region, codes in MAP_REGIONS_EXCEL.items():
        meteo2 = pd.read_excel(
            f"./data_FCD/IPMA/{codes[1]}-tx-tn-prec.xlsx",
            sheet_name=sheet_number,
        )
        meteo2.dropna(axis=0, how="any", inplace=True)

        #Obtain only years after 1986
        meteo2 = meteo2[meteo2.year > 1986]
        meteo2.set_index("year", inplace=True)
        meteo2.index.names = ["Region"]
        
        # use teh Describe that gives mean (index 2), max (index 7) and min (index 3)
        data_prec = meteo2.transpose().describe()
        index_names_arr = data_prec.index.values
        
        #Rename that column to region and drop the others
        index_names_arr[index_describe] = codes[0]
        for k_name in index_names_arr:
            if k_name != codes[0]:
                data_prec.drop(index=k_name, inplace=True)
        
        years_list = data_prec.keys()
        header = [years_list, [meteo_coln] * len(years_list)]
        data_prec.columns = header
        
        data_f = (
            data_prec
            if region == list(MAP_REGIONS_EXCEL.keys())[0]
            else pd.concat([data_f, data_prec])
        )
    return data_f


In [ ]:
def load_ipma():
    ipma = {
        type_data: add_extra_col(type_data, k)
        for k, type_data in enumerate(IPMA_COLUMNS)
    }
    ## Make all the concats at the same time to save time
    meteo = pd.concat([ipma[k] for k in IPMA_COLUMNS], axis=1)
    # meteo["T_range"] = meteo["TmaxA"] - meteo["TminA"]
    meteo= meteo.stack(level=0)
    meteo.index.names = ["Region", "Year"]
    return meteo.reindex(sorted(meteo.columns), axis=1)

In [ ]:
meteo = load_ipma()

## Added DTs
meteo['dT']=meteo['Tmax']-meteo['Tmin']
meteo['dTA']=meteo['TmaxA']-meteo['TminA']
meteo

If you look into this table we can see that Madeira (region 3) has some strange values for temperature, most of Tmin is zero

### 3.2. Correlation between our columns of Meterologic Data

Here we can see a high correlation between `Prec` and `Prec_max` and between the `Tmax`and `Tmin` and `TminA` we can also see some correlation

In [ ]:
meteo

In [ ]:
map_date(product, year_selected, colorscale="rainbow", meteo=True)

In [ ]:
def pairplot_data():
    meteo2 = meteo.reset_index()
    meteo2.set_index("Year")
    return meteo2 

In [ ]:
# A pairplot between all the meteo data that we are obtaining
plot_data = pairplot_data()
sns.pairplot(plot_data, hue="Region")
plt.show()

### 3.3. PCA for meteo data
Trying to study similarities between the years in terms of their metereological data

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca1=PCA(n_components=2)

meteo_scaled=StandardScaler().fit_transform(meteo_region)
principalComponents = pca1.fit_transform(meteo_scaled)
principalDf = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2'])

meteo_pca = pca1.transform(meteo_scaled)

pca1.explained_variance_ratio_

Our PCA only covers around 60% of the variance of the data which is not very good. Still, it may be interesting to try to visualize some similarities using these two principal components (with 3 principal components the PCA covers ~77%).

(Before adding the dT and dTA intervals to the meto data the PCA would cover 74% with 2 components and 86% with 3. The results obtained were similar).

In [ ]:
print(meteo_region.columns)
print(pca1.components_)

In [ ]:
# Plotting the results
labels=meteo_region.index
plt.scatter(meteo_pca[:,0], meteo_pca[:,1], label=labels)
for line, label in enumerate(labels):
     plt.text(meteo_pca[:,0][line]+0.2, meteo_pca[:,1][line], label, horizontalalignment='left', size=10, color='black', weight='semibold')

One of the things we can observe in the figure above is that the meteorological data of a certain year can be very different from the previous year.

## 4. Initial exploration of INE and IPMA data

### 4.1. Production a certain region

In [ ]:
filter_col = ["Trigo", "Amêndoa"]

year = range(1986, 2022)
def create_df_trg_amd(filter_col):
    df_alentejo_trig_amend = {
    }
    for k_col, col in enumerate(filter_col):
        if col not in df_alentejo_trig_amend:
            df_alentejo_trig_amend[col] = []
        for elem in data[col]["18"]:
            df_alentejo_trig_amend[col].append(elem)

    meteo_data = {}
    meteo_region=meteo.xs(18)
    col_to_draw = ["Prec_max"]
    for k_col, col in enumerate(col_to_draw):
        if col not in meteo_data:
            meteo_data[col] = []
        for elem in meteo_region[col]:
            meteo_data[col].append(elem)
        meteo_data["year"] = meteo_region[col].index
        
    for key in df_alentejo_trig_amend:
        df_alentejo_trig_amend[key] = MaxAbsScaler().fit_transform(np.array(df_alentejo_trig_amend[key]).reshape(-1, 1))
    return df_alentejo_trig_amend, meteo_data

In [ ]:
plot_data, plot_data2 = create_df_trg_amd(filter_col)
fig, (ax, ax2) = plt.subplots(2, 1, figsize=(16, 8))
for key in plot_data:
    ax.plot(year, plot_data[key], label=key, marker="o")
ax2.plot(plot_data2["year"], plot_data2["Prec_max"], color="red", marker="o")
ax2.set_xlim([1984, 2022])
ax.set_xlim([1984, 2022])
ax.set_ylabel("Quantity produced (kg/ha) Normalized with min max")
ax2.set_ylabel("Max Precipitation")
ax2.grid()
ax.grid()
ax.set_title("Quantity produced (kg/ha) by year in Alentejo (18)")
plt.xlabel("Years")
ax.legend()
plt.xticks(rotation="vertical")
plt.show()

### 4.2 Study of the Correlation between the Meteorologic Data and the production of Trigo for the North of Portugal

In [ ]:
meteo_region = meteo.xs(11)
data_region = data.xs("11")
index_to_plot= ["Trigo"]

index_min = list(meteo_region.index).index(1987)
index_max = list(meteo_region.index).index(2019)
meteo_region = meteo_region.iloc[range(index_min,index_max)]

index_min = list(data_region.index).index("1987")
index_max = list(data_region.index).index("2019")
data_region = data_region[index_to_plot].iloc[range(index_min,index_max)]


f,ax = plt.subplots(figsize=(12, 10))
cols = list(meteo_region.columns)
cols.append("Trigo")
cm = np.corrcoef(meteo_region.values.T, data_region.values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols, xticklabels=cols)
plt.show()
print(cols)

### 4.3. Quantity produced in comparison to temperature

In [ ]:
fig, (ax, ax2) = plt.subplots(2, 1, figsize=(16, 8))
year = range(1987, 2019)
ax.plot(year, data_region.values, label="Trigo", marker="o")
ax2.plot(year, meteo_region["Prec_min"], color="red", marker="o")
ax2.set_xlim([1984, 2022])
ax.set_xlim([1984, 2022])
ax.set_ylabel("Quantity produced")
ax2.set_ylabel("Min Precipitation")
ax2.grid()
ax.grid()
ax.set_title("Quantity of Trigo products by year in the north")
plt.xlabel("Years")
ax.legend()
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
meteo_region = meteo.xs(11)
data_region = data.xs("11")
index_to_plot= ["Trigo", "Amêndoa", "Ananás", "Trigo duro", "Trigo mole"]

index_min = list(meteo_region.index).index(1987)
index_max = list(meteo_region.index).index(2019)
meteo_region = meteo_region.iloc[range(index_min,index_max)]

index_min = list(data_region.index).index("1987")
index_max = list(data_region.index).index("2019")
data_region = data_region[index_to_plot].iloc[range(index_min,index_max)]


f,ax = plt.subplots(figsize=(12, 10))
cols = list(meteo_region.columns)
cols += index_to_plot
cm = np.corrcoef(meteo_region.values.T, data_region.values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols, xticklabels=cols)
plt.show()
# cols = cols.drop('Transported')
print(cols)

In [ ]:
meteo_region.index

### 4.4. Study of all correlations between production and meteo data

In [ ]:
def table_corre(max_year=2010, delta=0, region=None, min_cor=0.7):
    if delta != 0:
        print(f"Correlation values for the products and meteo data, with products with more {delta} years than product")
    if region is None:
        region = [11, 16, 18, 15, 3, 2, 17]
    print(f'{"Region":<10}{"Correlation value":>20} {"Meteo":>20} {"Product":>40}')
    for region_code in region:
        meteo_region = meteo.xs(region_code)
        data_region = data.xs(str(region_code))
        index_min = list(meteo_region.index).index(1987)
        index_max = list(meteo_region.index).index(max_year)
        meteo_region = meteo_region.iloc[range(index_min+delta,index_max+delta)]

        index_min = list(data_region.index).index("1987")
        index_max = list(data_region.index).index(str(max_year))
        data_region = data_region.iloc[range(index_min,index_max)]

        cm = np.corrcoef(meteo_region.values.T, data_region.values.T)
        list_nomes = list(meteo_region.columns)
        list_nomes.extend(data_region.columns)

        for k, sub_cm in enumerate(cm):
            if k <= 9: #Since 9 are the columns of meteo
                for k2, elem2 in enumerate(sub_cm):
                    if abs(elem2) > min_cor and k2 > 9:
                        print(f"{str(region_code)}{' ':>10}{elem2:>20}{list_nomes[k]:>20}{list_nomes[k2]:>40}")

In [ ]:
table_corre(delta=0, min_cor=0.75)

On the table above we can see that regions 11, 18 and 3 are the one with bigger correlations between metreology and products eficciency.

In [ ]:
plt.figure(figsize=(16, 8))
region_code = 18
max_year = 2010
meteo_region = meteo.xs(region_code)
data_region = data.xs(str(region_code))
products = ["Tomate para indústria", "Principais culturas para indústria"]
index_min = list(meteo_region.index).index(1987)
index_max = list(meteo_region.index).index(max_year)
meteo_region = meteo_region.iloc[range(index_min,index_max)]

index_min = list(data_region.index).index("1987")
index_max = list(data_region.index).index(str(max_year))
data_region = data_region[products].iloc[range(index_min,index_max)]

year = range(1987, max_year)
plt.plot(year, data_region.values, label=products, marker="o")
year = range(1987, max_year)
plt.ylabel("Quantity produced")
plt.legend()
ax2 = plt.twinx()
ax2.plot(year, meteo_region["dT"], color="red", marker="o", label="dt")
plt.ylabel("Temperature range")
plt.grid()
plt.title("Quantity of `Produtos para indústria` products by year in the North Region")
plt.xlabel("Years")
products.append("dT")
plt.legend()
plt.xticks(rotation="vertical")
plt.show()

## 5. Regplot
To explore how one variable in the meteorologic boletin affects one production in linear terms

In [ ]:
def meteo_data_region(region_code=11, min_year=1997, max_year=2010, products=None, delta=0):
    if products is None:
        products = ["Trigo"]
    meteo_region = meteo.xs(region_code)
    data_region = data.xs(str(region_code))
    index_min = list(meteo_region.index).index(min_year)
    index_max = list(meteo_region.index).index(max_year)
    meteo_region = meteo_region.iloc[range(index_min,index_max)]

    index_min = list(data_region.index).index(str(min_year))
    index_max = list(data_region.index).index(str(max_year))
    data_region = data_region[products].iloc[range(index_min+delta,index_max+delta)]
    year = range(1987, max_year)
    return year, data_region, meteo_region

In [ ]:
def reg_plot_products(meteo_x="dT", products="Ameixa", max_year=2016, region=18, min_year=1988):
     year, data_region, meteo_region = meteo_data_region(region_code=region, min_year=min_year, max_year=max_year, products=[products])
     data_region.index = data_region.index.map(str)
     meteo_region.index = meteo_region.index.map(str)
     df = meteo_region.join(data_region)
     cm = np.corrcoef(df[meteo_x], df[products])
     print(f"They have a correlation of {cm[0, 1]*100:.2f}")
     plt.figure(figsize=(16, 8))
     sns.regplot(data=df, x=meteo_x, y=products)
     for line in range(df.shape[0]):
          plt.text(df[meteo_x][line], df[products][line], df.index[line], horizontalalignment='left', size=10, color='black', weight='semibold')

In [ ]:
reg_plot_products()

In [ ]:
reg_plot_products(meteo_x="Prec_max", products="Azeitona de mesa", max_year=2009, region=18, min_year=1992)

In [ ]:
reg_plot_products(meteo_x="dT", products="Nespera", max_year=1997, region=11, min_year=1987)

In [ ]:
reg_plot_products(meteo_x="dT", products="Tomate para indústria", max_year=2018, region=18, min_year=1987)

## Machine Learning
We previously saw a few examples of regression plots, we will associate them to Machine learning models in the chapter. This way any user can, by simply placing a few values, obtain the estimation of the production by hectar to that product if the trends continue.

We decided to use 6 regression models and compare the results between them:
- Linear Regression - Uses a linear model to try to predict the results
- BayesianRidge - BayesianRidge estimates a probabilistic model of the regression proble . The prior for the coefficient is given by a spherical Gaussian.
- PassiveAggressiveRegressor - They are similar to the Perceptron in that they do not require a learning rate. However, contrary to the Perceptron, they include a regularization parameter C.
- TweedieRegressor - Generalized Linear Model with a Tweedie distribution.
- Ridge Regression - uses a L2 it tries to reduce the value of the variables with low value.
- Elastic Net - L1  and L2 priors for Regulation and can also make the weight of a uninportant variable equal to zero.


We have used a StandardScaler so that we could use Machine learning, without it, the higher values variables would gain more weight than others.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

### 6.1. Regression Models

In [ ]:
products=["Tomate para indústria"]
region=18
min_year=1987
max_year=2018
year_to_study = 2017 

In [ ]:
def train_model(products, max_year, region, min_year):
    year, data_region, meteo_region = meteo_data_region(region_code=region, min_year=min_year, max_year=max_year, products=products)

    scaler = StandardScaler()
    scaler.fit(meteo_region)
    X = pd.DataFrame(scaler.transform(meteo_region), columns=meteo_region.columns)

    scaler_y = StandardScaler()
    scaler_y.fit(data_region)
    y = pd.DataFrame(scaler_y.transform(data_region), columns=data_region.columns)

    lin_reg = LinearRegression().fit(X, y)
    bay_reg = BayesianRidge().fit(X, y)
    pas_reg = PassiveAggressiveRegressor().fit(X, y)
    las_reg = TweedieRegressor(max_iter=50000).fit(X, y)
    rid_reg = Ridge(max_iter=50000).fit(X, y)
    ela_reg = ElasticNet().fit(X, y)
    reg = [lin_reg, bay_reg, pas_reg, las_reg, rid_reg, ela_reg]
    return reg, X, y, scaler, scaler_y, data_region

def obtain_meteo(scaler, region_code=11, year_to_study=1997, use_scaler=False):
    meteo_region = meteo.xs(region_code)
    index_min = list(meteo_region.index).index(year_to_study)
    index_max = list(meteo_region.index).index(year_to_study+1)
    meteo_region = meteo_region.iloc[range(index_min,index_max)]
    return meteo_region


In [ ]:
reg, X, y, scaler, scaler_y, y_no_norm = train_model(products, max_year, region, min_year)
x_pred = obtain_meteo(region_code=region, year_to_study=year_to_study, scaler=scaler)
x_pred2 = pd.DataFrame(scaler.transform(x_pred), columns=meteo_region.columns)

In [ ]:
import contextlib
regressions=['Linear', "BayesianRidge", "PassiveAggressive", "TweedieRegressor", "Ridge", "ElasticNet"]
preds = [regre.predict(x_pred) for regre in reg]

for k_regre, regre in enumerate(reg):
    print(f"The {regressions[k_regre]} Regression predicted {preds[k_regre]}")
    print(f"{regre.get_params(deep=False)}")
    with contextlib.suppress(AttributeError):
        print(f"{regre.n_iter_}")

We have decided to leave all the models in their default format this way a user can always change the products, and obtain new models with a relatively good hyper-parametrization.

We were only interested in one specific product then we could do a hyper parameter search on it, to improve that model even further.

In [ ]:
preds2 = pd.DataFrame(scaler_y.inverse_transform(preds), columns=products)
preds2

In [ ]:
y2 = pd.DataFrame(scaler_y.inverse_transform(y), columns=products)

In [ ]:
plt.scatter(regressions, preds2)
Val=y2[products].loc[y2.index[-1]] # Actual value of the production
plt.hlines(Val,-0.5,len(preds2)-0.5,colors='black',label='Actual Production')
plt.ylabel('Production')
plt.legend()
plt.show()

Above we can see the expected production according to the different regression models used, while the black line represents the actual production. This way we can see and compare the results between the different models.

Tweedie seems to perform very well in most cases; ElasticNet does great when there's high correlation between the production and one of the meteo features.

PassiveAggressive seems to do exceptionally bad (sometimes even negative production numbers!!!)

### 6.2. Explore predictions with made up meteorological data
Change any parameter from the X_pred and see how the models predictions change

In [ ]:
x_pred2 = obtain_meteo(region_code=region, year_to_study=year_to_study, scaler=scaler, use_scaler=False)
x_pred2["Prec_max"]=300
x_pred2 = pd.DataFrame(scaler.transform(x_pred2), columns=meteo_region.columns)
x_pred2

In [ ]:
preds_new_original = [regre.predict(x_pred2) for regre in reg]
preds_new = pd.DataFrame(scaler_y.inverse_transform(preds_new_original), columns=products)
preds_new

In [ ]:
plt.scatter(regressions,preds2)
plt.scatter(regressions,preds_new)
plt.ylabel('Production')
plt.show()

### 6.3. Coefficient study for the various models

In [ ]:
# plotting the coefficient score
# fig, ax = plt.subplots(figsize =(20, 10))
color = ["red","green","blue", "black","orange","purple","beige","brown","gray","cyan","magenta"]
for k in range(6):
    word = regressions[k]
    print(word)
    ax1 = plt.subplot(2, 3, k+1)
    ax1.scatter(X.columns, reg[k].coef_, color=color[k])
    ax1.legend(fancybox=True, shadow=True, labels=word)
plt.style.use('ggplot')
plt.show()

Note: the elastic net usually considers all coeficients as 0 unless there is a high correlation between one of the meteo features and the product. (For example 'Tomate para indústria' has a 82% correlation with dT)

In [ ]:
# plotting the coefficient score
fig, ax = plt.subplots()

for k in range(6):
    ax.scatter(X.columns, reg[k].coef_,color=color[k])
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
          ncol=3, fancybox=True, shadow=True, labels=regressions)
plt.style.use('ggplot')
plt.show()

As we can see in the both images above, the elastic net only used dT,  and Tmin and dT were the variables where the coeficients more vary per model.